# Competitive Analysis of Restaurants in Toronto

### Reading List of Neighborhodds from table in dataframe 

In [5]:
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',skiprows=0)[0]

In [6]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
# Get names of indexes for which column Borough has value 'Not assigned'
indexNames = df[df['Borough'] == 'Not assigned' ].index
indexNames

Int64Index([  0,   1,   7,  10,  15,  16,  19,  24,  25,  28,  29,  33,  34,
             35,  37,  38,  42,  43,  44,  51,  52,  53,  60,  61,  62,  69,
             70,  71,  78,  79,  87,  88,  96,  97, 101, 105, 106, 110, 115,
            118, 119, 123, 124, 125, 127, 128, 131, 132, 133, 134, 136, 137,
            140, 141, 145, 146, 149, 150, 154, 155, 158, 159, 161, 162, 163,
            164, 166, 167, 170, 171, 172, 173, 174, 175, 176, 177, 179],
           dtype='int64')

In [8]:
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Replacing data without neighborhood name with Borough's name

In [9]:
df.loc[df['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df['Borough']

In [10]:
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## Reading the postal code based Longitude/Latitude data from csv file

In [11]:
#Reading the postal code based Longitude/Latitude data from csv file

!wget -q -O 'postal_data.csv'  http://cocl.us/Geospatial_data
df_ll = pd.read_csv('postal_data.csv')

In [12]:
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Renaming columns. This will enable merging with previous dataframe
df_ll.columns=['Postal code','Latitude','Longitude']

In [14]:
#Merge the two dataframes
df_alldata = pd.merge(df, df_ll[['Postal code','Latitude', 'Longitude']], on='Postal code')
df_alldata.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [16]:
#Import all necessary
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!conda install -c conda-forge --no-deps altair --yes
#!conda install -c conda-forge --no-deps vincent --yes

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge geopy=1.19.0 --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

#!conda install -c conda-forge --no-deps folium=0.5.0 --yes
!pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 7.3MB/s eta 0:00:011
Libraries imported.


In [17]:
#Getting the number of boroughs and neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [19]:
#Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [45]:
#Create a map of Toronto with neighborhoods superimposed on top
# create map of Toronto using latitude and longitude values
#!pip install folium
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_alldata['Latitude'], df_alldata['Longitude'], df_alldata['Borough'], df_alldata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### The following cell contains credentials for Foursquare

In [21]:
CLIENT_ID = 'ZMI0AMJUBEU5PZ5OGIK2QPV2XS0MNKLKYRHDDEBWGMWGXPCW' # your Foursquare ID
CLIENT_SECRET = 'QOMNRZHAD2T4TGWSUVWB0VFZZGHUVUBANUJ1GFFF1EMSH4VV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZMI0AMJUBEU5PZ5OGIK2QPV2XS0MNKLKYRHDDEBWGMWGXPCW
CLIENT_SECRET:QOMNRZHAD2T4TGWSUVWB0VFZZGHUVUBANUJ1GFFF1EMSH4VV


In [22]:
#Checking data for one neighborhood only
neighborhood_latitude = df_alldata.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_alldata.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_alldata.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
#code to run the above function on each neighborhood and create a new dataframe
toronto_venues = getNearbyVenues(names=df_alldata['Neighborhood'],
                                   latitudes=df_alldata['Latitude'],
                                   longitudes=df_alldata['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(2167, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,8,8,8,8,8,8
Bathurst Manor / Wilson Heights / Downsview North,21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
Bedford Park / Lawrence Manor East,24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56
Birch Cliff / Cliffside West,4,4,4,4,4,4
Brockton / Parkdale Village / Exhibition Place,22,22,22,22,22,22
Business reply mail Processing CentrE,17,17,17,17,17,17


In [29]:
#Analyze each neighborhood
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## All Toronto Venues with theier Neighborhood names

In [30]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
6,Regent Park / Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
7,Regent Park / Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
8,Regent Park / Harbourfront,43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
9,Regent Park / Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [31]:
toronto_venues.shape

(2167, 7)

## Finding all Unique Venue Categories

In [32]:
#Get unique Venue categories
toronto_venues['Venue Category'].unique() 

array(['Park', 'Food & Drink Shop', 'Hockey Arena', 'Coffee Shop',
       'Portuguese Restaurant', 'Intersection', 'Bakery',
       'Distribution Center', 'Spa', 'Breakfast Spot', 'Restaurant',
       'Historic Site', 'Pub', 'Farmers Market', 'Chocolate Shop',
       'Dessert Shop', 'Performing Arts Venue', 'French Restaurant',
       'Café', 'Mexican Restaurant', 'Event Space', 'Yoga Studio',
       'Ice Cream Shop', 'Asian Restaurant', 'Shoe Store', 'Theater',
       'Cosmetics Shop', 'Electronics Store', 'Bank', 'Beer Store',
       'Hotel', 'Health Food Store', 'Antique Shop', 'Boutique',
       'Furniture / Home Store', 'Vietnamese Restaurant',
       'Clothing Store', 'Accessories Store', 'Miscellaneous Shop',
       'Italian Restaurant', 'Beer Bar', 'Creperie',
       'Arts & Crafts Store', 'Burrito Place', 'Diner', 'Hobby Shop',
       'Wings Joint', 'Fried Chicken Joint', 'Burger Joint', 'Juice Bar',
       'Sandwich Place', 'Gym', 'College Auditorium', 'Bar',
       'College 

## Seggregating the venues which are eateries

In [33]:
#Manually pick food related venues. Add them to a list. 
food_list = ['Food & Drink Shop', 'Coffee Shop', 'Portuguese Restaurant', 'Bakery', 'Breakfast Spot', 'Restaurant', 'French Restaurant', 'Café', 'Mexican Restaurant', 'Asian Restaurant',  'Health Food Store', 'Vietnamese Restaurant', 'Burrito Place', 'Diner', 'Wings Joint', 'Fried Chicken Joint',
'Burger Joint', 'Sandwich Place', 'Vegetarian / Vegan Restaurant', 'Fast Food Restaurant', 'Caribbean Restaurant', 'Japanese Restaurant', 
'Pizza Place', 'Ramen Restaurant', 'Thai Restaurant', 'Steakhouse', 'American Restaurant', 'Sushi Restaurant', 'Middle Eastern Restaurant',
'Modern European Restaurant', 'Seafood Restaurant', 'Chinese Restaurant', 'Ethiopian Restaurant', 'Greek Restaurant', 'Dim Sum restaurant', 'BBQ Joint', 'Food Truck', 'New American Restaurant', 'German Restaurant', 'Comfort Food Restaurant', 'Moroccan Restaurant',
'Belgian Restaurant',  'Bagel Shop', 'Eastern European Restaurant', 'Gourmet Shop', 'Indian Restaurant', 'Korean Restaurant', 'Falafel Restaurant', 'Salad Place', 'Deli / Bodega', 'Hakka Restaurant', 'Mediterranean Restaurant', 'Food Court', 'Noodle House', 'Colombian Restaurant', 'Brazilian Restaurant', 'Gluten-free Restaurant', 'Mac & Cheese Joint', 'Tapas Restaurant', 'Cafeteria', 'Empanada Restaurant', 
'Latin American Restaurant', 'Indonesian Restaurant','Cajun / Creole Restaurant', 'Dumpling Restaurant', 'Doner Restaurant', 'Filipino Restaurant', 'Molecular Gastronomy Restaurant', 'Taiwanese Restaurant', 'Snack Place',  'Theme Restaurant', 'Afghan Restaurant', 'Polish Restaurant']

In [34]:
food_list

['Food & Drink Shop',
 'Coffee Shop',
 'Portuguese Restaurant',
 'Bakery',
 'Breakfast Spot',
 'Restaurant',
 'French Restaurant',
 'Café',
 'Mexican Restaurant',
 'Asian Restaurant',
 'Health Food Store',
 'Vietnamese Restaurant',
 'Burrito Place',
 'Diner',
 'Wings Joint',
 'Fried Chicken Joint',
 'Burger Joint',
 'Sandwich Place',
 'Vegetarian / Vegan Restaurant',
 'Fast Food Restaurant',
 'Caribbean Restaurant',
 'Japanese Restaurant',
 'Pizza Place',
 'Ramen Restaurant',
 'Thai Restaurant',
 'Steakhouse',
 'American Restaurant',
 'Sushi Restaurant',
 'Middle Eastern Restaurant',
 'Modern European Restaurant',
 'Seafood Restaurant',
 'Chinese Restaurant',
 'Ethiopian Restaurant',
 'Greek Restaurant',
 'Dim Sum restaurant',
 'BBQ Joint',
 'Food Truck',
 'New American Restaurant',
 'German Restaurant',
 'Comfort Food Restaurant',
 'Moroccan Restaurant',
 'Belgian Restaurant',
 'Bagel Shop',
 'Eastern European Restaurant',
 'Gourmet Shop',
 'Indian Restaurant',
 'Korean Restaurant',
 

## Removing non-eatery venues from Venue list

In [35]:
#Keep the rows from toronto_venues which have venue in food_list
toronto_food_venues = toronto_venues[toronto_venues['Venue Category'].isin(food_list)] 

In [36]:
toronto_food_venues.shape

(1028, 7)

In [37]:
toronto_food_venues.reset_index(inplace=True,drop=True)
toronto_food_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
1,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
2,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
3,Regent Park / Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
4,Regent Park / Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
5,Regent Park / Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
6,Regent Park / Harbourfront,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
7,Regent Park / Harbourfront,43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
8,Regent Park / Harbourfront,43.654260,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
9,Regent Park / Harbourfront,43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop


In [120]:
toronto_food_venues.shape

(1048, 7)

## Get count of eateries in each neighborhood

In [38]:
#This gives the count of Food venues in each neighborhood.
df_venueCount=toronto_food_venues.groupby(['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']).size().reset_index(name='Venue Count')

In [39]:
df_venueCount

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Count
0,Agincourt,43.794200,-79.262029,3
1,Alderwood / Long Branch,43.602414,-79.543484,4
2,Bathurst Manor / Wilson Heights / Downsview North,43.754328,-79.442259,10
3,Bayview Village,43.786947,-79.385975,3
4,Bedford Park / Lawrence Manor East,43.733283,-79.419750,15
5,Berczy Park,43.644771,-79.373306,25
6,Birch Cliff / Cliffside West,43.692657,-79.264848,1
7,Brockton / Parkdale Village / Exhibition Place,43.636847,-79.428191,10
8,Business reply mail Processing CentrE,43.662744,-79.321558,4
9,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,1


In [40]:
#Sorting results based on venue count
df_sorted=df_venueCount.sort_values(by=['Venue Count','Neighborhood'])
df_sorted.reset_index(inplace=True,drop=True)
df_sorted.rename(columns = {'Neighborhood Latitude':'Latitude', 'Neighborhood Longitude':'Longitude'}, inplace = True) 

### OUTPUT NO. 1 : Displaying the sorted list of neighborhoods with their eatery count. The neighborhoods with lowest competitions are listed first.

In [43]:
df_sorted

,Neighborhood,Latitude,Longitude,Venue Count
0,Birch Cliff / Cliffside West,43.692657,-79.264848,1
1,CN Tower / King and Spadina / Railway Lands / ...,43.628947,-79.394420,1
2,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,1
3,Downsview,43.728496,-79.495697,1
4,East Toronto,43.685347,-79.338106,1
5,Forest Hill North & West,43.696948,-79.411307,1
6,Hillcrest Village,43.803762,-79.363452,1
7,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,1
8,Malvern / Rouge,43.806686,-79.194353,1
9,North Park / Maple Leaf Park / Upwood Park,43.713756,-79.490074,1


## Mapping the data on map

### OUTPUT 2 : The different neighborhoods are marked with different colours to indicate the level of competition in the area

In [44]:
#Visualize and create map 

# create map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
venueColor='';
for lat, lon, poi, venuecount in zip(df_sorted['Latitude'], df_sorted['Longitude'], df_sorted['Neighborhood'], df_sorted['Venue Count']):
    if(venuecount<2):
        venueColor='green'
    elif(venuecount<5):
        venueColor='lightgreen'
    elif(venuecount<10):
        venueColor='yellow'
    elif(venuecount<20):
        venueColor='orange'
    elif(venuecount>20):
        venueColor='red'
    label = folium.Popup(str(poi) + ' -Restaurant Count: ' + str(venuecount), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=venueColor,
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)
       
map_toronto

## End of Project